1. Hoàn tất riêng lẻ các phần. Các phần có input - output: phần text
2. Hoàn tất riêng lẻ các phần. Các phần có input - output: text vào file Excel và Audio vào đúng thư mục . 
3. Ghép nối các luồng tuần tự trong .ipynb
4. Ghép và loại bỏ các phần thừa. 
5. Các tối ưu riêng lẻ từng phần về sau.  


- Ver 1: Đẩy nhanh sản phẩm chia nhau hoặc outsource 
    - 1.1 tách nhỏ bài toán Prompting 
    - 1.2 xử lý input output chuẩn định dạng automatic
- Ver 2: Làm tất các phần, đóng gói 1 luồng. 
    - 2.1 tích hợp công nghệ bên kia sao cho chuẩn công nghệ bên mình. (thêm gen content, gen audio, gen task 3). 
    - 2.2 tối ưu phần cũ (đóng gói thành các hàm). 
    - 2.3 ghép nối thông input và output dạng cơ bản nhất của đầu ra (excel nhưng chưa cần đủ các cột của nhập liệu). 

## 1. Hoàn tất riêng lẻ các phần. Các phần có input - output: phần text

Để không thêm ```JSON ở đầu output:
```
    # Output: Return a JSON object with the following structure:
    Ensure that the output is a valid JSON object without any additional formatting or keywords (such as JSON).
```
- Tiêu chuẩn JSON: Định dạng JSON chính thức không bao gồm bất kỳ từ khóa hoặc tiền tố nào trước dữ liệu thực tế.
- Parsing: Các thư viện xử lý JSON như json.loads() trong Python mong đợi dữ liệu JSON thuần túy, không có bất kỳ tiền tố nào.
- Tính nhất quán: Giữ cho output nhất quán với định dạng JSON tiêu chuẩn giúp đảm bảo tính tương thích với các công cụ và thư viện khác nhau.
- Độ rõ ràng: JSON tự nó đã là một định dạng được định nghĩa rõ ràng, không cần thêm bất kỳ nhãn hoặc định danh nào.

### Set Up LLMs

In [7]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_completion(prompt, model="gpt-4o", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [8]:
print(os.getenv("OPENAI_API_KEY"))

sk-proj-zxUCZnIlYGiviDsNYEh8T3BlbkFJlKwiHfS9NfjWeTEHB3MB


### Set Up Audio

```cmd 
curl -L -X POST "http://103.253.20.13:25010/api/text-to-speech" -H "Content-Type: application/json" -d "{\"text\": \"hello. Its me.Can you tell me?\",\"voice\": \"en-AU-WilliamNeural\",\"speed\": 1}" --output "1.mp3"
```

```
curl -L -X POST "http://103.253.20.13:25010/api/text-to-speech" -H "Content-Type: application/json" -d "{\"text\": \"Chapter 2: What is the purpose of life? Translator: Luan Nguyen. Please keep listening, everyone. Some of you have been to this retreat before, while others are new to it. When you listen to the lecture, try to keep your mouth closed naturally and pay attention to what's being said. In Buddhism, there are two important terms that describe the journey of a practitioner: wisdom and unwisdom. So, what do they mean? Wisdom is about truly understanding things as they are. It's also known as right view or enlightenment. On the other hand, unwisdom means having an incorrect understanding of things and phenomena. This is also called wrong view, false thinking, or delusion. When you observe or examine any phenomenon, there are two types of knowledge to consider: the wisdom taught by the Buddha and the unwisdom of ordinary people. The goal of Buddhist practice is to overcome unwisdom and cultivate wisdom instead. Why is this the case? Well, it's because the way we live is determined by what we know. Let me give you an example: imagine a two-year-old child who sees a red-hot coal burning in the fireplace. Even though the child may recognize the object as a coal, they don't fully understand the danger of touching it with their bare hands. So, if there's no one around to stop them, they might grab the coal and burn their hands badly. But for adults, who have a better understanding of the danger of a burning coal, they wouldn't touch it with their bare hands or feet. By living according to this knowledge, they can avoid suffering from burns or injuries. Our knowledge determines how we live. Children with limited knowledge might touch a burning coal and suffer, while adults with a deeper understanding of risks will avoid touching it with bare hands, preventing unnecessary pain. In the same way, if someone's understanding of daily life is based on delusion, confusion, or wrong views, it's like being a child who doesn't understand the danger of touching a burning coal. This leads to conflicts with reality, and their life becomes filled with sorrow and suffering. If a person has wisdom, right view, and insight, and understands the truth about things and phenomena, their life won't be in conflict with reality. They won't experience grief or sorrow. As I mentioned earlier regarding wisdom and unwisdom, anyone can feel the crunchy texture when chewing on a sesame cracker - whether they are an Arhat, a wise person, or an ordinary individual. However, there are two types of knowledge that arise from this experience. One is the knowledge of unwisdom, which assumes that the brittle texture of the sesame crackers is the actual material nature of it, and that crispiness is an inherent quality of the sesame crackers. This type of knowledge is considered unwisdom, delusion, and illusion in Buddhism. The second understanding is that brittleness is a sensation created in one's mind, rather than being a property of matter. It arises from the teeth coming into contact with the sesame crackers, and is impermanent, ownerless, and non-possessive. This knowledge is wisdom. Therefore, not only should one examine the brittleness of phenomena, but all phenomena should be examined through these two types of knowledge. Why should we strive for wisdom? To attain it, live by it, to end confusion, gain insight, live in harmony with reality, and free ourselves from suffering.\",\"voice\": \"en-AU-WilliamNeural\",\"speed\": 1}" --output "1.mp3"
```

## Hàm gen 4 paragraphs và audio => Duyệt qua từng hàng của input để gen và lưu vào 1 sheet excel. 

In [5]:
import requests

def text_to_speech(text, voice, speed, output_path):
    url = "http://103.253.20.13:25010/api/text-to-speech"
    headers = {
        "Content-Type": "application/json"
    }
    payload = {
        "text": text,
        "voice": voice,
        "speed": speed
    }
    
    response = requests.post(url, json=payload, headers=headers)
    
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            f.write(response.content)
        print(f"File saved to {output_path}")
    else:
        print(f"Error: {response.status_code} - {response.text}")

# Example usage
text_to_speech("hello. Its me.Can you tell me?", "en-AU-WilliamNeural", 1, "1.mp3")


File saved to 1.mp3


#### def gen_content_Dang1Listening

In [6]:
def gen_content_Dang1Listening(given_words, topic):
    """
    Input: givern_words (gồm 6-9 cụm từ), 1 topic
    Output: 4 đoạn văn, 4 list gồm các cụm, 
    """


    prompt = f"""
    Act as an English Teacher. Your students are A2 English learners (CEFR).

    Task for you: Write a short story, which includes 4 sections, with the content of each section based on given words and the topic.

    Requirement for the task: 
    - These four sections must relate to each other and create a complete story that is easy for A2 level. 
    - The total length of four sections must not exceed 100 words.
    - Four sections must be equal.
    - Each section must contain at least 2-3 given words.
    - You must use easy-to-understand vocabularies, suitable for A2 level.
    - 
    Return a JSON object with the following structure:
    Ensure that the output is a valid JSON object without any additional formatting or keywords (such as JSON).
    {{
        "paragraph1": "content of section1",
        "list1": ["word1", "word2", "word3"],
        "paragraph2": "content of section2",
        "list2": ["word4", "word5", "word6"],
        "paragraph3": "content of section3",
        "list3": ["word7", "word8", "word9"],
        "paragraph4": "content of section4",
        "list4": ["word10", "word11", "word12"]
    }}

    The following are given words for each section and the topic:
    Given words: {", ".join(given_words)}
    Topic: {topic}
    """

    response = get_completion(prompt)
    # return response


    # Validate and format the JSON response
    try:
        # Parse the response to ensure it's valid JSON
        json_response = json.loads(response)
        
        # Re-serialize to ensure proper JSON formatting
        return json.dumps(json_response, ensure_ascii=False, indent=None)
    except json.JSONDecodeError as e:
        print(f"Error: The generated response is not valid JSON. Error details: {str(e)}")
        return None


##### given_words example

In [7]:
import json
# Example usage
given_words = ["apple", "banana", "cherry", "date", "elderberry", "fig", "grape", "honeydew"]
topic = "A Fruit Adventure"

result = gen_content_Dang1Listening(given_words, topic)
print(result)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************B3MB. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [52]:
json_response = json.loads(result)
print(type(result))

<class 'str'>


#### def gen_content_AND_audio_Dang1Listening 

In [2]:

def gen_content_AND_audio_Dang1Listening(given_words, topic, audio_folder_output):
    """
    Input: givern_words (gồm 6-9 cụm từ), 1 topic, audio_folder_output
    Output: 4 đoạn văn, 4 list gồm các cụm, và 4 audio đánh số từ 1 đến 4 được lưu vào thư mục output_path
    """

    # Tạo 1 thư mục tên "audio_output" nếu chưa tồn tại, nếu tồn tại rồi thì bỏ qua
    
    if not os.path.exists(audio_folder_output):
        os.makedirs(audio_folder_output)
        print(f"Đã tạo thư mục '{audio_folder_output}'")
    else:
        print(f"Thư mục '{audio_folder_output}' đã tồn tại")

    # Gen audio dựa vào hàm text_to_speech ở bên trên
    response = gen_content_Dang1Listening(given_words, topic)
    paragraph_dict = json.loads(response)  # Convert the string to a dictionary

    # paragraphs = [
    #     paragraph_dict.get("paragraph1"),
    #     paragraph_dict.get("paragraph2"),
    #     paragraph_dict.get("paragraph3"),
    #     paragraph_dict.get("paragraph4")
    # ]
    
    paragraphs = [paragraph_dict.get(f"paragraph{i}") for i in range(1, 5)]
    phrase_lists = [paragraph_dict.get(f"list{i}") for i in range(1, 5)]

    for i, paragraph in enumerate(paragraphs, start=1):
        if paragraph:
            output_path = f"{audio_folder_output}/{i}.mp3"
            text_to_speech(text=paragraph, voice="en-AU-WilliamNeural", speed=1, output_path=output_path)

    return paragraphs, phrase_lists

In [84]:
# gen_content&audio_Dang1Listening
gen_content_AND_audio_Dang1Listening(given_words=given_words, topic=topic, audio_folder_output="new_output")

Thư mục 'new_output' đã tồn tại
File saved to new_output/1.mp3
File saved to new_output/2.mp3
File saved to new_output/3.mp3
File saved to new_output/4.mp3


(['Tom and Anna went on a fruit adventure. They found an apple tree and picked some apples. They also saw a banana tree nearby.',
  'Next, they walked to a cherry orchard. The cherries were red and sweet. They ate some and put more in their basket.',
  'They continued their adventure and found a fig tree. The figs were ripe and delicious. They also saw some elderberries.',
  'Finally, they reached a grapevine. The grapes were juicy. They also found a honeydew melon. It was a fun fruit adventure!'],
 [['apple', 'banana', 'cherry'],
  ['date', 'elderberry', 'fig'],
  ['grape', 'honeydew', 'apple'],
  ['banana', 'cherry', 'date']])

#### def input_2_gen_content_AND_audio_2_excel: Duyệt qua từng hàng từ file Input, run hàm gen_content_audio để return paragraph và phrases_lists, sau đó lưu từng output vào file output excel
Không nên def gen_content_AND_audio_AND_excel_Dang1Listening (cho 1 topic, 6-0 cụm, sau đó output được lưu vào excel, nhưng nếu như thế thì khi duyệt qua các hàng của 1 file excel để lấy topic và cụm => lại thành 1 loạt excel rời rạc)

Input: 

```
| Chủ đề                         | Cụm                                                              |
|--------------------------------|------------------------------------------------------------------|
| Talk about you working day     | half past eight am, seven am, noon, checking emails, answering phone calls, scheduling meetings, drink coffee, chat with colleagues, take a walk |
| Talk about you working day (2) | at five pm, at a quarter to six pm, when my tasks are completed, satisfied, accomplished, productive |

```

In [85]:
import os
import pandas as pd
from openpyxl import Workbook
from typing import List, Tuple



def input_2_gen_content_AND_audio_2_excel(input_file_path: str, output_folder: str) -> Tuple[List[str], List[str], str]:
    """
    Process the input file, generate content and audio, and save the results to an Excel file.

    Args:
        input_file_path (str): Path to the input Excel file.
        output_folder (str): Path to the folder where audio files and Excel file will be saved.

    Returns:
        Tuple[List[str], List[str], str]: A tuple containing lists of paragraphs, audio paths, and the path to the output Excel file.
    
    Prompt: 
    - Đọc file input file 

            ```
            | Chủ đề                         | Cụm                                                              |
            |--------------------------------|------------------------------------------------------------------|
            | Talk about you working day     | half past eight am, seven am, noon, checking emails, answering phone calls, scheduling meetings, drink coffee, chat with colleagues, take a walk |
            | Talk about you working day (2) | at five pm, at a quarter to six pm, when my tasks are completed, satisfied, accomplished, productive |

```
    - Tìm đến sheet "Input bài nghe"
    - Duyệt qua từng hàng, trong từng hàng lấy topic ở cột: "Chủ đề", lấy các cụm ở cột "Cụm"
    Mục tiêu return ra được `given_words`, `topic`, 
        - Sau đó lắp vào hàm def gen_content_AND_audio_Dang1Listening(given_words, topic, audio_folder_output) 
        để nhận về paragraph list chứa 4 đoạn paragraph1, 2, 3, 4 , và phrase_list chứa các cụm đã dùng để tạo 4 đoạn đó, 

        paragraphs = [paragraph_dict.get(f"paragraph{i}", "") for i in range(1, 5)]
        phrase_lists = [paragraph_dict.get(f"list{i}", "") for i in range(1, 5)]
        - Lưu audio vào thư mục row_id (với row_id là số hàng từ input_file)
        - Lưu 1 file excel : 
            1. Bạn tạo 1 file excel nếu file chưa tạo, hoặc viết đè lên 1 file đã tạo. đặt tên "output_dang1_LISTENING"
            2. Tạo 1 sheet tên là "mix_listen_content"
            3. Sheet gồm các cột thông tin như trong ảnh
            'id', 'content_part_1', 'content_part_2', 'content_part_3', 'content_part_4',
               'audio_part_1', 'audio_part_2', 'audio_part_3', 'audio_part_4',
               'vocab_part_1', 'vocab_part_2', 'vocab_part_3', 'vocab_part_4'
    - Ở mỗi row_id tôi sẽ gen 4 audio, 4 audio này được lưu vào thư mục tên "row_id", 
    thư mục output_folder sẽ chứa các folder row_id này và file excel nhé

    """
    os.makedirs(output_folder, exist_ok=True)
    output_excel_file = f"{output_folder}/output_dang1_LISTENING.xlsx".replace(os.sep, '/')
    df = pd.read_excel(input_file_path, sheet_name="Input bài nghe")

    results = []
    for index, row in df.iterrows():
        row_id = index + 1
        topic = row["Chủ đề"]
        given_words = row["Cụm"].split(", ")

        row_folder = f"{output_folder}/row_{row_id}".replace(os.sep, '/')
        os.makedirs(row_folder, exist_ok=True)

        paragraphs, phrase_lists = gen_content_AND_audio_Dang1Listening(given_words, topic, row_folder)
        audio_paths = [f"{row_folder}/{i}.mp3".replace(os.sep, '/') for i in range(1, 5)]

        results.append({
            'id': row_id,
            'content_part_1': paragraphs[0],
            'content_part_2': paragraphs[1],
            'content_part_3': paragraphs[2],
            'content_part_4': paragraphs[3],
            'audio_part_1': audio_paths[0],
            'audio_part_2': audio_paths[1],
            'audio_part_3': audio_paths[2],
            'audio_part_4': audio_paths[3],
            'vocab_part_1': ', '.join(phrase_lists[0]),
            'vocab_part_2': ', '.join(phrase_lists[1]),
            'vocab_part_3': ', '.join(phrase_lists[2]),
            'vocab_part_4': ', '.join(phrase_lists[3])
        })

    result_df = pd.DataFrame(results)

    workbook = Workbook()
    sheet = workbook.active
    sheet.title = "mix_listen_content"
    sheet.append([
        'id', 'content_part_1', 'content_part_2', 'content_part_3', 'content_part_4',
        'audio_part_1', 'audio_part_2', 'audio_part_3', 'audio_part_4',
        'vocab_part_1', 'vocab_part_2', 'vocab_part_3', 'vocab_part_4'
    ])

    for _, result in result_df.iterrows():
        new_row = [
            result['id'],
            result['content_part_1'],
            result['content_part_2'],
            result['content_part_3'],
            result['content_part_4'],
            result['audio_part_1'],
            result['audio_part_2'],
            result['audio_part_3'],
            result['audio_part_4'],
            result['vocab_part_1'],
            result['vocab_part_2'],
            result['vocab_part_3'],
            result['vocab_part_4']
        ]
        sheet.append(new_row)

    workbook.save(output_excel_file)
    print(f"Details have been written to Excel file '{output_excel_file}'")

    return result_df, output_excel_file

# Example usage
input_file_path = r"D:\OneDrive - Hanoi University of Science and Technology\ITE10-DS&AI-HUST\Learn&Task\PRODUCT_THECOACH\Task2_ContentGeneration\ĐÓNG_GÓI_1_FLOW\input_TOPIC_and_PHRASES.xlsx"
output_folder = "./out_folder"

result_df, output_file_path = input_2_gen_content_AND_audio_2_excel(input_file_path, output_folder)


Thư mục './out_folder/row_1' đã tồn tại
File saved to ./out_folder/row_1/1.mp3
File saved to ./out_folder/row_1/2.mp3
File saved to ./out_folder/row_1/3.mp3
File saved to ./out_folder/row_1/4.mp3
Thư mục './out_folder/row_2' đã tồn tại
File saved to ./out_folder/row_2/1.mp3
File saved to ./out_folder/row_2/2.mp3
File saved to ./out_folder/row_2/3.mp3
File saved to ./out_folder/row_2/4.mp3


PermissionError: [Errno 13] Permission denied: './out_folder/output_dang1_LISTENING.xlsx'

#### def ex1_DIEN_VAO_CHO_TRONG_Dang1Listening => Sau đó: duyệt qua từng hàng để lưu từng output lưu vào file excel 

In [71]:
import json

def ex1_DIEN_VAO_CHO_TRONG_Dang1Listening(input_text, terms_list):
    """
    Return: 1 câu đục ở 2 chỗ là từ đơn (thuộc các cụm đã cho ở đoạn 1). Đáp án gồm 2 đáp án đơn đúng và 4-5 đáp án đơn sai.
    """


    prompt = f"""
    # Designation(S) - Role(R) - Contextualization (C):
    You are a content creation expert in creating exercises for language learning.

    Your task: Create a fill-in-the-blank exercise based on:
    - the given input text: "{input_text}"
    - and terms from the given list: {terms_list}.

    -------------------
    # Instructions (I):
    1. Get 'selected_term' and get 'question': 
    !!! IMPORTANT NOTE: We treat compound nouns and compound adjectives with hyphens as SINGLE WORD. (some SINGLE WORD examples: a, an, the, Nam, 15-minute, work, now, ...)
    Select 2 single words (MUST BE SINGLE WORDS) from 2 terms of the terms in the `terms_list`
    and replace them (MUST BE 2 SINGLE WORDS) with "__" to create a question.

    2. Create 'answers_list' answer: Create 7 answer choices: including the 2 replaced words and 5 wrong words (wrong words should seem plausible but incorrect).
    MANDATORY: ALL 7 answer choices MUST BE SINGLE WORDS. 

    3. Return a JSON object with the keys: input_text, selected_term, question, words.

    -------------------
    # Output: Return a JSON object with the following structure:
    Ensure that the output is a valid JSON object without any additional formatting or keywords.

    {{
        "input_text": "The original input text `input_text`",
        "selected_term": ["selected term 1", "selected term 2"],
        "question": "The input text with the two SINGLE WORDS replaced by __",
        "answers_list": ["replaced_single_word_1", "replaced_single_word_2", "wrong_single_word_1", "wrong_single_word_2", "wrong_single_word_3", "wrong_single_word_4", "wrong_single_word_5"]
    }}


    # Example of expected output format:

    {{
        "input_text": "Anna works in an office. She handles the paperwork and does repetitive tasks every day. Even though the salary is low, she tries to learn to enjoy her job.",
        "selected_term": ["the paperwork", "to enjoy her job"],
        "question": "Anna works in an office. She handles the __ and does repetitive tasks every day. Even though salary is low, she tries to learn to __ her job.",
        "answers_list": ["paperwork", "enjoy", "tasks", "office", "work", "learn", "handle"]
    }}
    """

    response = get_completion(prompt, temperature=0)
    # return response

    # Để an toàn thì response nên trả ra theo cách này để loads luôn 
    # Validate JSON response
    try:
        return response
    except json.JSONDecodeError:
        print("Error: The generated response is not valid JSON. Please check the output.")
        return None



In [127]:
response = gen_content_Dang1Listening(given_words, topic)

# Convert the string to a dictionary
paragraph_dict = json.loads(response)

# Extract "paragraph1" and "list1" from response
paragraph1 = paragraph_dict.get("paragraph1")
list1 = paragraph_dict.get("list1")
response1 = ex1_DIEN_VAO_CHO_TRONG_Dang1Listening(paragraph1, list1)
print(response1)



{
    "input_text": "Anna and Ben went to the market. They bought an apple and a banana.",
    "selected_term": ["apple", "banana"],
    "question": "Anna and Ben went to the market. They bought an __ and a __.",
    "answers_list": ["apple", "banana", "cherry", "market", "went", "bought", "they"]
}


In [128]:
import pandas as pd
# def gồm các tham số file_path, sheet_name, column name

# Define the file path
file_path = r'D:\OneDrive - Hanoi University of Science and Technology\ITE10-DS&AI-HUST\Learn&Task\PRODUCT_THECOACH\Task2_ContentGeneration\ĐÓNG_GÓI_1_FLOW\gen_content&audio_Dang1Listening.xlsx'

# Load the Excel file
excel_data = pd.ExcelFile(file_path)

# Access the sheet "Đã sửa content"
sheet_name = 'Input bài nghe'
df = pd.read_excel(excel_data, sheet_name=sheet_name)

# Extract the "Exercise" Answer column
expected_answer_4_paragraph = df['4 đoạn']

# Khởi tạo danh sách các từ điển để lưu các hàng dữ liệu
data = []

# Duyệt qua từng paragraph trong expected_answer_4paragraph
for paragraph in expected_answer_4_paragraph: 
    # type paragraph: object
    print(type(paragraph[1]))
    # # Convert the string to a dictionary
    paragraph_dict = json.loads(paragraph)

    # Extract with key: "paragraph1" and "list1" 
    paragraph1 = paragraph_dict.get("paragraph1")
    list1 = paragraph_dict.get("list1")

    # Print the extracted values
    print("paragraph1:", paragraph1)
    print("list1:", list1)
    print("\n")

    response = ex1_DIEN_VAO_CHO_TRONG_Dang1Listening(paragraph1, list1)
    print(response, type(response1)) # str
    # response = '{"input_text": "Emma wakes up at seven am. She gets ready and makes a cup of coffee. She drinks coffee and eats toast for breakfast.", "selected_term": ["seven", "coffee"], "question": "Emma wakes up at __ am. She gets ready and makes a cup of coffee. She drinks __ and eats toast for breakfast.", "words": ["seven", "coffee", "toast", "ready", "cup", "drinks", "breakfast"]}'
    
    response_dict = json.loads(response)

    data.append({
        "question": response_dict["question"],
        "paragraph": response_dict["input_text"],
        # "words": ", ".join(response1["words"])  # Chuyển danh sách các từ thành chuỗi
        "answer": "\n".join(response_dict["answers_list"])  # Chuyển danh sách các từ thành chuỗi với xuống dòng
    })

# Tạo DataFrame từ dữ liệu
df = pd.DataFrame(data)

# Ghi DataFrame ra file Excel
output_file_path = 'ex1_Dang1Listening_output.xlsx'
df.to_excel(output_file_path, index=False)

print(f"Data has been successfully written to {output_file_path}")


<class 'str'>
paragraph1: Emma wakes up at seven am. She gets ready and makes a cup of coffee. She drinks coffee and eats toast for breakfast.
list1: ['seven am', 'drink coffee']


{
    "input_text": "Emma wakes up at seven am. She gets ready and makes a cup of coffee. She drinks coffee and eats toast for breakfast.",
    "selected_term": ["seven am", "drink coffee"],
    "question": "Emma wakes up at __ am. She gets ready and makes a cup of coffee. She __ coffee and eats toast for breakfast.",
    "answers_list": ["seven", "drink", "ready", "makes", "cup", "eats", "toast"]
} <class 'str'>
<class 'str'>
paragraph1: John starts his work day at nine am. He checks his emails and plans his tasks. By noon, he feels productive and happy.
list1: ['productive']


{
    "input_text": "John starts his work day at nine am. He checks his emails and plans his tasks. By noon, he feels productive and happy.",
    "selected_term": ["emails", "productive"],
    "question": "John starts his work day at

#### def ex2_CHON_DAP_AN_PHU_HOP_Dang1Listening

In [72]:
def ex2_CHON_DAP_AN_PHU_HOP_Dang1Listening(input_text, terms_list):
    """
    Return: 1 câu hỏi từ đoạn 2, và đáp án đúng là 1 cụm trong các cụm từ đoạn 2 (3 cụm sai)
    """

    prompt = f"""
    # Designation(S) - Role(R):
    You are a content creation expert specializing in creating exercises for language learning.

    # Contextualization (C):
    Your task: Create a multiple-choice exercise with 4 options based on the given text: {input_text}
    and a randomly selected term from the provided list: {terms_list}.
    -----------------------
    # Instructions (I):

    ** Step 1: Select one term ('selected_term') from the given list: `terms_list`.
    ** Step 2: Extract a sentence ('sentence') from the input text that contains the 'selected_term'.
    ** Step 3: Create a 5W1H question ('question') based on the extracted sentence.
    ** Step 4: Provide four answer options ('answers_list'):
        - (a) The correct answer containing the 'selected_term'.
        - (b, c, d) Plausible but incorrect answers.
    ** Step 5: Return the output in JSON format.

    ------------------
    # Output: Return a JSON object with the following keys: selected_term, sentence, question, answers_list.
    Ensure that the output is a valid JSON object without any additional formatting or keywords (such as JSON).

    {{
        "selected_term": "example_term",
        "sentence": "example_sentence",
        "question": "example_question",
        "answers_list": [
            "correct_answer",
            "incorrect_answer_1",
            "incorrect_answer_2",
            "incorrect_answer_3"
        ]
    }}

    --------------------
    # Example:

    {{
        "selected_term": "collaborative environment",
        "sentence": "In their collaborative environment, creativity and innovation flourished.",
        "question": "What kind of environment allowed creativity and innovation to flourish?",
        "answers_list": [
            "Collaborative environment",
            "Competitive environment",
            "Quiet environment",
            "Stressful environment"
        ]
    }}

    # Self-Evaluation - Self-Reflection - Responsibility, Creative, Add Emotion - Evaluate():
    1. Clear previous training data.
    2. Assume the role of a learner given the input text.
    3. Review the question and answer list carefully.
    4. If the first answer in 'answers_list' is correct, proceed; if not, re-evaluate and start over.

    """

    response = get_completion(prompt, temperature=0)
    try:
        return response
    except json.JSONDecodeError:
        print("Error: The generated response is not valid JSON. Please check the output.")
        return None


In [10]:
response = gen_content_Dang1Listening(given_words, topic)

# Convert the string to a dictionary
paragraph_dict = json.loads(response)

# Extract "paragraph1" and "list1" from response
paragraph2 = paragraph_dict.get("paragraph2")
list2 = paragraph_dict.get("list2")
response2 = ex2_CHON_DAP_AN_PHU_HOP_Dang1Listening(paragraph2, list2)
print(response2)



{
    "selected_term": "elderberry",
    "sentence": "Next, they found a cherry tree.",
    "question": "What did they find next?",
    "answers_list": [
        "A cherry tree",
        "An elderberry bush",
        "A fig tree",
        "A date palm"
    ]
}


In [12]:

import pandas as pd
# def gồm các tham số file_path, sheet_name, column name

# Define the file path
file_path = r'D:\OneDrive - Hanoi University of Science and Technology\ITE10-DS&AI-HUST\Learn&Task\PRODUCT_THECOACH\Task2_ContentGeneration\ĐÓNG_GÓI_1_FLOW\gen_content&audio_Dang1Listening.xlsx'

# Load the Excel file
excel_data = pd.ExcelFile(file_path)

# Access the sheet "Đã sửa content"
sheet_name = 'Input bài nghe'
df = pd.read_excel(excel_data, sheet_name=sheet_name)

# Extract the "Exercise" Answer column
expected_answer_4_paragraph = df['4 đoạn']

# Khởi tạo danh sách các từ điển để lưu các hàng dữ liệu
data = []

# Duyệt qua từng paragraph trong expected_answer_4paragraph
for paragraph in expected_answer_4_paragraph: 

    # # Convert the string to a dictionary
    paragraph_dict = json.loads(paragraph)

    # Extract with key: "paragraph1" and "list1" 
    paragraph2 = paragraph_dict.get("paragraph2")
    list2 = paragraph_dict.get("list2")

    # Print the extracted values
    print("paragraph2:", paragraph2)
    print("list2:", list2)
    print("\n")

    response_str = ex2_CHON_DAP_AN_PHU_HOP_Dang1Listening(paragraph2, list2)
    print(response_str, type(response_str)) # str

    response_dict = json.loads(response_str)

    data.append({
        "question": response_dict["question"],
        # "words": ", ".join(response_dict["words"])  # Chuyển danh sách các từ thành chuỗi
        "answer": "\n".join([answer.strip() for answer in response_dict["answers_list"]]) # Chuyển danh sách các từ thành chuỗi với xuống dòng
    })


# Tạo DataFrame từ dữ liệu
df = pd.DataFrame(data)

# Ghi DataFrame ra file Excel
output_file_path = 'ex2_Dang1Listening_output.xlsx'
df.to_excel(output_file_path, index=False)

print(f"Data has been successfully written to {output_file_path}")



paragraph2: At half past eight am, Emma starts work from home. She begins by checking emails. She also schedules meetings for the day.
list2: ['half past eight am', 'checking emails', 'scheduling meetings']


{
    "selected_term": "checking emails",
    "sentence": "She begins by checking emails.",
    "question": "What does Emma start her workday with?",
    "answers_list": [
        "Checking emails",
        "Making coffee",
        "Reading reports",
        "Scheduling meetings"
    ]
} <class 'str'>
paragraph2: At five pm, John finishes his main tasks. He feels satisfied with what he has done. He takes a short break with some tea.
list2: ['at five pm', 'satisfied']


{
    "selected_term": "satisfied",
    "sentence": "He feels satisfied with what he has done.",
    "question": "How does John feel about what he has done?",
    "answers_list": [
        "Satisfied",
        "Disappointed",
        "Confused",
        "Indifferent"
    ]
} <class 'str'>
paragraph2: Jane liked that

#### def ex3_NOI_DE_TRA_LOI_Dang1Listening

In [73]:


def ex3_NOI_DE_TRA_LOI_Dang1Listening(input_text, terms_list): 
    """
    Input: 
    - input_text: a string containing the text to base the exercise on.
    - terms_list: a list of terms to choose from for creating the exercise. 
    <PARAMETERS ARE NOT NECESSARY, INCLUDED TO SYNCHRONIZE THE FUNCTIONS>

    Output: JSON format: 1 câu hỏi cho cả đoạn 3 - NÓI ĐỂ TRẢ LỜI - trong nhập liệu có prompt để check phần trả lời
    
    """

    prompt = f"""
    Based on the content of provided paragraph, create one question using 5W1H method. You must return one question only, and make sure the question is suitable for B1 English learners (CEFR).
    The following is content of provided paragraph:
    <paragraph>: {input_text}
    Return: a valid JSON object without any additional formatting or keywords (such as JSON)
    Example: 
    {{
        "question": "Where are you from?"
    }}
    """
    response = get_completion(prompt, temperature=0)
    return response


In [45]:
response = gen_content_Dang1Listening(given_words, topic)

# Convert the string to a dictionary
paragraph_dict = json.loads(response)

# Extract "paragraph" and "list" from response
paragraph3 = paragraph_dict.get("paragraph3")
list3 = paragraph_dict.get("list3")
response3 = ex3_NOI_DE_TRA_LOI_Dang1Listening(paragraph3, list3)
print(response3)



{
    "question": "What had Tom never tried before?"
}


In [48]:

import pandas as pd
# def gồm các tham số file_path, sheet_name, column name

# Define the file path
file_path = r'D:\OneDrive - Hanoi University of Science and Technology\ITE10-DS&AI-HUST\Learn&Task\PRODUCT_THECOACH\Task2_ContentGeneration\ĐÓNG_GÓI_1_FLOW\gen_content&audio_Dang1Listening.xlsx'
# Load the Excel file
excel_data = pd.ExcelFile(file_path)

# Access the sheet "Đã sửa content"
sheet_name = 'Input bài nghe'
df = pd.read_excel(excel_data, sheet_name=sheet_name)

# Extract the "Exercise" Answer column
expected_answer_4_paragraph = df['4 đoạn']

# Khởi tạo danh sách các từ điển để lưu các hàng dữ liệu
data = []

# Duyệt qua từng paragraph trong expected_answer_4paragraph
for paragraph in expected_answer_4_paragraph: 

    # # Convert the string to a dictionary
    paragraph_dict = json.loads(paragraph)

    # Extract with key: "paragraph1" and "list1" 
    paragraph3 = paragraph_dict.get("paragraph3")
    list3 = paragraph_dict.get("list3")

    # Print the extracted values
    print("paragraph3:", paragraph3)
    print("list3:", list3)
    print("\n")

    response_str = ex3_NOI_DE_TRA_LOI_Dang1Listening(paragraph3, list3)
    print(response_str, type(response_str)) # str

    response_dict = json.loads(response_str)

    data.append({
        "question": response_dict["question"],
        "paragraph": paragraph3
        # "answer": "\n".join([answer.strip() for answer in response_dict["answers_list"]]) # Chuyển danh sách các từ thành chuỗi với xuống dòng
    })


# Tạo DataFrame từ dữ liệu
df = pd.DataFrame(data)

# Ghi DataFrame ra file Excel
output_file_path = 'ex3_Dang1Listening_output.xlsx'
df.to_excel(output_file_path, index=False)

print(f"Data has been successfully written to {output_file_path}")

paragraph3: Before noon, Emma is busy answering phone calls from clients. She chats with her colleagues during a short break.
list3: ['noon', 'answering phone calls', 'chat with colleagues']


{
    "question": "What does Emma do before noon?"
} <class 'str'>
paragraph3: At a quarter to six pm, John reviews his schedule. He makes sure all important tasks are completed. He feels accomplished and content.
list3: ['at a quarter to six pm', 'accomplished']


{
    "question": "What does John do at a quarter to six pm?"
} <class 'str'>
paragraph3: One day, Jane missed a deadline. She felt upset and knew she needed to fix it.
list3: ['missed a deadline']


{
    "question": "Why did Jane feel upset?"
} <class 'str'>
paragraph3: One great thing about Max's job was the fewer meetings. That gave him time to work hard on his tasks and be confident in his work.
list3: ['fewer meetings', 'work hard', 'to be confident']


{
    "question": "Why did Max like his job?"
} <class 'str'>
paragraph3: In 

#### def ex4_DIEN_PHAN_CON_THIEU_Dang1Listening

In [98]:


def ex4_DIEN_PHAN_CON_THIEU_Dang1Listening(input_text, terms_list):
    """
    Input: paragraph4, list 4 terms
    Output: JSON format: Gồm đoạn 4 bị đục 1 chỗ là 1 cụm ở đoạn 4 và đáp án là DẠNG ĐÚNG của cụm bị đục. 
    """

    prompt = f"""
# Designation(S) - Role(R):
You are a content creation expert in creating exercises for language learning.

# Contextualization (C):
Your task: Create a fill-type exercise based on the given {input_text} text
and 1 random term from several terms in the given {terms_list} list.

# Instructions (I):
** Step 1: Choose 'selected_term': Choose 1 Choose only 1 term of the terms in the `terms_list`.  
** Step 2: Find 'Dạng đúng của selected_term in `input_text`': exactly dạng chia động từ của `selected_term` in `input_text` 
(example: `selected_term` là: 'go to school' => Dạng đúng của selected_term in `input_text` chẳng hạn: 'going to school'.
 `selected_term` là: 'the marketing' => Dạng đúng của selected_term in `input_text`` chẳng hạn: 'marketing'. )
** Step 3: Provide `question`: fill-type exercise bằng cách replace `selected_term` in `input_text` ban đầu with "__" to create a question. 
** Step 4: Provide `answer`: List contain 1 element the 'correct_form_of_selected_term' identified in Step 2.

# Output: ONLY RETURN JSON format with key: selected_term, sentence, fill_in_the_blank_question, answers_list
Ensure that the output is a valid JSON object without any additional formatting or keywords (such as JSON).
{{  
    "selected_term": "selected_term",
    "Dạng đúng của selected_term": Dạng đúng của selected_term
    "sentence": "extracted sentence",
    "question": "Your fill-type question here", 
    "answers_list": ["Dạng chia động từ đúng của `selected_term`"]
}}


# Example:

### Dạng đúng của cụm từ
{{
    "selected_term": "go to the beach",
    "sentence": "During the summer, they love going to the beach every weekend to relax and enjoy the sun.",
    "question": "During the summer, they love __ every weekend to relax and enjoy the sun.",
    "answers_list": ["going to the beach"]
}}

### Dạng đúng của động từ 
{{
    "selected_term": "The marketing",
    "sentence": "Each person in the team has a different role. Liam handles finance, Sara manages marketing, and Tom focuses on sales. They all work together to help the company succeed.",
    "question": "Each person in the team has a different role. Liam handles finance, Sara manages __, and Tom focuses on sales. They all work together to help the company succeed.",
    "answers_list": ["The marketing"]
}}



# Self Evaluate - Self-Reflection - Responsibility, Creative, Add Emotion - Evaluate(): 
1. Xoá hết dữ liệu cũ đã được đào tạo. 
2. Bạn sẽ đóng vai là người học. Bạn được cho đoạn văn tại `input_text`. input_text: {input_text}
3. Bạn được cho `fill_in_the_blank_question`.
4. Bạn sẽ tự đọc kỹ đoạn văn, đọc kỹ câu hỏi, để tìm từ chính xác điền vào chỗ trống. 
5. Nếu câu trả lời TRÙNG VỚI `selected_term` => thì câu trả lời của bạn đúng rồi nhé. 
Nếu sai, bạn tự kiểm điểm lại mình, và BUỘC PHẢI QUAY LẠI ĐẦU TIÊN CỦA PROMPT ĐỂ KIỂM TRA LẠI . 

"""


    response = get_completion(prompt, temperature=0)
    return response

In [99]:
given_words = "I feel satisfied and accomplished. It was a productive day at work."
topic = "satisfied, accomplished, productive"

response = gen_content_Dang1Listening(given_words, topic)

# Convert the string to a dictionary
paragraph_dict = json.loads(response)

# Extract "paragraph" and "list" from response
paragraph4 = paragraph_dict.get("paragraph4")
list4 = paragraph_dict.get("list4")
response4 = ex4_DIEN_PHAN_CON_THIEU_Dang1Listening(paragraph4, list4)
print(response4)



{
    "selected_term": "relax",
    "Dạng đúng của selected_term": "relax",
    "sentence": "Now, I can relax at home.",
    "question": "Now, I can __ at home.",
    "answers_list": ["relax"]
}


In [94]:

import pandas as pd
# def gồm các tham số file_path, sheet_name, column name

# Define the file path
file_path = r'D:\OneDrive - Hanoi University of Science and Technology\ITE10-DS&AI-HUST\Learn&Task\PRODUCT_THECOACH\Task2_ContentGeneration\ĐÓNG_GÓI_1_FLOW\gen_content&audio_Dang1Listening.xlsx'

# Load the Excel file
excel_data = pd.ExcelFile(file_path)

# Access the sheet "Đã sửa content"
sheet_name = 'Input bài nghe'
df = pd.read_excel(excel_data, sheet_name=sheet_name)

# Extract the "Exercise" Answer column
expected_answer_4_paragraph = df['4 đoạn']

# Khởi tạo danh sách các từ điển để lưu các hàng dữ liệu
data = []

# Duyệt qua từng paragraph trong expected_answer_4paragraph
for paragraph in expected_answer_4_paragraph: 

    # # Convert the string to a dictionary
    paragraph_dict = json.loads(paragraph)

    # Extract with key: "paragraph1" and "list1" 
    paragraph4 = paragraph_dict.get("paragraph4")
    list4 = paragraph_dict.get("list4")

    # Print the extracted values
    print("paragraph:", paragraph4)
    print("list:", list4)
    print("\n")

    response_str = ex4_DIEN_PHAN_CON_THIEU_Dang1Listening(paragraph4, list4)
    print(response_str, type(response_str)) # str

    response_dict = json.loads(response_str)

    data.append({
        "question": response_dict["question"],
        # "words": ", ".join(response_dict["words"])  # Chuyển danh sách các từ thành chuỗi
        "answer": "\n".join([answer.strip() for answer in response_dict["answers_list"]]) # Chuyển danh sách các từ thành chuỗi với xuống dòng
    })


# Tạo DataFrame từ dữ liệu
df = pd.DataFrame(data)

# Ghi DataFrame ra file Excel
output_file_path = 'ex4_Dang1Listening_output.xlsx'
df.to_excel(output_file_path, index=False)

print(f"Data has been successfully written to {output_file_path}")

paragraph: After lunch, Emma likes to take a walk in the park. It helps her relax and feel fresh for the rest of the day.
list: ['take a walk']


{
    "selected_term": "take a walk",
    "Dạng đúng của selected_term": "taking a walk",
    "sentence": "After lunch, Emma likes to take a walk in the park. It helps her relax and feel fresh for the rest of the day.",
    "question": "After lunch, Emma likes __ in the park. It helps her relax and feel fresh for the rest of the day.",
    "answers_list": ["taking a walk"]
} <class 'str'>
paragraph: When his tasks are completed, John logs off his computer. He relaxes and thinks about his productive day at work.
list: ['when my tasks are completed', 'productive']


{
    "selected_term": "productive",
    "Dạng đúng của selected_term": "productive",
    "sentence": "When his tasks are completed, John logs off his computer. He relaxes and thinks about his productive day at work.",
    "question": "When his tasks are completed, John logs off his

### RUN ALL:
- H1: Output từng phần: gen bài xong =>  ex1, ex2, ex3, ex4
    - Đối với hàm gen def input_2_gen_content_AND_audio_2_excel chỉ cần return: df0 để có thể gộp 
- H2: Output cuối: xong hết => return output: gen bài, ex1, 2, 3, 4. 
    -> 

#### Run all from file_after_gen_content (với việc nhận file sau gen 4 đoạn từ a Hoàng)

In [87]:
import pandas as pd
import json

def process_excel(file_path, sheet_name, column_name, processing_func):
    """
    Parameters: 
        - processing_func là 1 hàm: ... Khi gọi processing_func(paragraph_dict)
    Input là file của a Hoàng,tìm đến cột Input bài nghe, để lấy paragraph1, 2, 3, 4 và list1, 2, 3, 4 để gen
    Định dạng như này:
        {
        "paragraph1":"John wakes up at seven am and gets ready for work. He starts work at half past eight am. The first thing he does is drink coffee.",
        "list1":["half past eight am", "seven am", "drink coffee"],

        "paragraph2":"John begins his tasks by checking emails and answering phone calls. By noon, he is busy scheduling meetings.",
        "list2":["checking emails", "answering phone calls", "noon", "scheduling meetings"],

        "paragraph3":"In the afternoon, John likes to take a walk and chat with colleagues when he feels tired. It helps him stay productive.",
        "list3":["take a walk", "chat with colleagues", "productive"],

        "paragraph4":"At five pm, John finishes his work. When his tasks are completed, he feels satisfied and accomplished. He leaves the office at a quarter to six pm.",
        "list4":["at five pm", "when my tasks are completed", "satisfied", "accomplished", "at a quarter to six pm"]
        } 
    
    """

    # Load the Excel file
    excel_data = pd.ExcelFile(file_path)
    
    # Access the specified sheet
    df = pd.read_excel(excel_data, sheet_name=sheet_name)
    
    # Extract the specified column
    expected_answer_4_paragraph = df[column_name]
    
    # Khởi tạo danh sách các từ điển để lưu các hàng dữ liệu
    data = []
    
    # Duyệt qua từng paragraph trong expected_answer_4_paragraph
    for paragraph in expected_answer_4_paragraph:
        # Convert the string to a dictionary
        paragraph_dict = json.loads(paragraph)
        
        # Gọi hàm xử lý tùy chỉnh
        response_str = processing_func(paragraph_dict)
        response_dict = json.loads(response_str)
    

        # Determine which paragraph to use based on the processing function
        if processing_func.__name__ == 'process_paragraph1':
            paragraph_text = paragraph_dict.get("paragraph1", "")
        elif processing_func.__name__ == 'process_paragraph2':
            paragraph_text = paragraph_dict.get("paragraph2", "")
        elif processing_func.__name__ == 'process_paragraph3':
            paragraph_text = paragraph_dict.get("paragraph3", "")
        elif processing_func.__name__ == 'process_paragraph4':
            paragraph_text = paragraph_dict.get("paragraph4", "")
        else:
            paragraph_text = ""  # Default case if the function name doesn't match

        data.append({
            "question": response_dict["question"],
            "paragraph": paragraph_text,  # paragraph_dict.get("paragraph1, 2, 3, 4"),     # paragraph1, 2, 3, 4 tương ứng các đoạn luôn
            "answer": "\n".join([answer.strip() for answer in response_dict.get("answers_list", [])])
        })
    
    # Tạo DataFrame từ dữ liệu
    output_df = pd.DataFrame(data)
    
    return output_df

# Define the file path and sheet name
file_path = r'D:\OneDrive - Hanoi University of Science and Technology\ITE10-DS&AI-HUST\Learn&Task\PRODUCT_THECOACH\Task2_ContentGeneration\ĐÓNG_GÓI_1_FLOW\gen_content&audio_Dang1Listening.xlsx'
sheet_name = 'Input bài nghe'
column_name = '4 đoạn'

# Define the processing functions
def process_paragraph1(paragraph_dict):
    paragraph1 = paragraph_dict.get("paragraph1")
    list1 = paragraph_dict.get("list1")
    return ex1_DIEN_VAO_CHO_TRONG_Dang1Listening(paragraph1, list1)

def process_paragraph2(paragraph_dict):
    paragraph2 = paragraph_dict.get("paragraph2")
    list2 = paragraph_dict.get("list2")
    return ex2_CHON_DAP_AN_PHU_HOP_Dang1Listening(paragraph2, list2)

def process_paragraph3(paragraph_dict):
    paragraph3 = paragraph_dict.get("paragraph3")
    list3 = paragraph_dict.get("list3")
    return ex3_NOI_DE_TRA_LOI_Dang1Listening(paragraph3, list3)

def process_paragraph4(paragraph_dict):
    paragraph4 = paragraph_dict.get("paragraph4")
    list4 = paragraph_dict.get("list4")
    return ex4_DIEN_PHAN_CON_THIEU_Dang1Listening(paragraph4, list4)

# Process each type of paragraph and store in DataFrames
df1 = process_excel(file_path, sheet_name, column_name, process_paragraph1)
df2 = process_excel(file_path, sheet_name, column_name, process_paragraph2)
df3 = process_excel(file_path, sheet_name, column_name, process_paragraph3)
df4 = process_excel(file_path, sheet_name, column_name, process_paragraph4)

# Write all DataFrames to a single Excel file with different sheets
output_file_path = 'combined_Dang1Listening_output.xlsx'
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    df1.to_excel(writer, sheet_name='Ex1_listen_fill_choose', index=False)
    print(f"Sheet 'Ex1_listen_fill_choose' has been successfully written.")

    df2.to_excel(writer, sheet_name='Ex2_listen_choice', index=False)
    print(f"Sheet 'Ex2_listen_choice' has been successfully written.")
    
    df3.to_excel(writer, sheet_name='Ex3_listen_speak', index=False)
    print(f"Sheet 'Ex3_listen_speak' has been successfully written.")

    df4.to_excel(writer, sheet_name='Ex4_listen_fill_type', index=False)
    print(f"Sheet 'Ex4_listen_fill_type' has been successfully written.")

print(f"Data has been successfully written to {output_file_path}")


Sheet 'Ex1_listen_fill_choose' has been successfully written.
Sheet 'Ex2_listen_choice' has been successfully written.
Sheet 'Ex3_listen_speak' has been successfully written.
Sheet 'Ex4_listen_fill_type' has been successfully written.
Data has been successfully written to combined_Dang1Listening_output.xlsx


#### Ver 1 - Run all from file_after_gen_content - CƯỜNG
FROM file: "input_TOPIC_and_PHRASES.xlsx"  => 

In [7]:
import os
import pandas as pd
from openpyxl import Workbook
from typing import List, Tuple



def input_2_gen_content_AND_audio_2_excel(input_file_path: str, output_folder: str) -> Tuple[pd.DataFrame, str]:
    """
    Input file: 

        - Đọc file input file 

            ```
            | Chủ đề                         | Cụm                                                              |
            |--------------------------------|------------------------------------------------------------------|
            | Talk about you working day     | half past eight am, seven am, noon, checking emails, answering phone calls, scheduling meetings, drink coffee, chat with colleagues, take a walk |
            | Talk about you working day (2) | at five pm, at a quarter to six pm, when my tasks are completed, satisfied, accomplished, productive |


    Process the input file, generate content and audio, and save the results to an Excel file.

    Args:
        input_file_path (str): Path to the input Excel file.
        output_folder (str): Path to the folder where audio files and Excel file will be saved.

    Returns:
        Tuple[pd.DataFrame, str]: A tuple containing the resulting DataFrame and the path to the output Excel file.
    """


    os.makedirs(output_folder, exist_ok=True)
    output_excel_file = f"{output_folder}/output_dang1_LISTENING.xlsx".replace(os.sep, '/')
    df = pd.read_excel(input_file_path, sheet_name="Input bài nghe")

    results = []
    for index, row in df.iterrows():
        row_id = index + 1
        topic = row["Chủ đề"]
        given_words = row["Cụm"].split(", ")

        row_folder = f"{output_folder}/row_{row_id}".replace(os.sep, '/')
        os.makedirs(row_folder, exist_ok=True)

        paragraphs, phrase_lists = gen_content_AND_audio_Dang1Listening(given_words, topic, row_folder)
        audio_paths = [f"{row_folder}/{i}.mp3".replace(os.sep, '/') for i in range(1, 5)]

        results.append({
            'id': row_id,
            'content_part_1': paragraphs[0],
            'content_part_2': paragraphs[1],
            'content_part_3': paragraphs[2],
            'content_part_4': paragraphs[3],
            'audio_part_1': audio_paths[0],
            'audio_part_2': audio_paths[1],
            'audio_part_3': audio_paths[2],
            'audio_part_4': audio_paths[3],
            'vocab_part_1': ', '.join(phrase_lists[0]),
            'vocab_part_2': ', '.join(phrase_lists[1]),
            'vocab_part_3': ', '.join(phrase_lists[2]),
            'vocab_part_4': ', '.join(phrase_lists[3])
        })

    result_df = pd.DataFrame(results)

    workbook = Workbook()
    sheet = workbook.active
    sheet.title = "mix_listen_content"
    sheet.append([
        'id', 'content_part_1', 'content_part_2', 'content_part_3', 'content_part_4',
        'audio_part_1', 'audio_part_2', 'audio_part_3', 'audio_part_4',
        'vocab_part_1', 'vocab_part_2', 'vocab_part_3', 'vocab_part_4'
    ])

    for _, result in result_df.iterrows():
        new_row = [
            result['id'],
            result['content_part_1'],
            result['content_part_2'],
            result['content_part_3'],
            result['content_part_4'],
            result['audio_part_1'],
            result['audio_part_2'],
            result['audio_part_3'],
            result['audio_part_4'],
            result['vocab_part_1'],
            result['vocab_part_2'],
            result['vocab_part_3'],
            result['vocab_part_4']
        ]
        sheet.append(new_row)

    workbook.save(output_excel_file)
    print(f"Details have been written to Excel file '{output_excel_file}'")

    return result_df, output_excel_file

# Example usage
input_file_path = r"D:\OneDrive - Hanoi University of Science and Technology\ITE10-DS&AI-HUST\Learn&Task\PRODUCT_THECOACH\Task2_ContentGeneration\ĐÓNG_GÓI_1_FLOW\input_TOPIC_and_PHRASES.xlsx"
output_folder = "./out_folder"

result_df, output_file_path = input_2_gen_content_AND_audio_2_excel(input_file_path, output_folder)


Thư mục './out_folder/row_1' đã tồn tại


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************B3MB. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [103]:
import pandas as pd
import json

def process_excel(file_path, sheet_name, processing_func, content_col, vocab_col):
    """
    - Input: là 
        | id | content_part_1                                                                 | content_part_2                                                                 | content_part_3                                                                 | content_part_4                                                | audio_part_1              | audio_part_2              | audio_part_3              | audio_part_4              | vocab_part_1                                      | vocab_part_2                                           | vocab_part_3                                      | vocab_part_4                        |
        |----|--------------------------------------------------------------------------------|--------------------------------------------------------------------------------|--------------------------------------------------------------------------------|---------------------------------------------------------------|---------------------------|---------------------------|---------------------------|---------------------------|---------------------------------------------------|----------------------------------------------------------|---------------------------------------------------|-------------------------------------|
        | 1  | I wake up at seven am and get ready for work. By half past eight am, I am at my desk. | In the morning, I start by checking emails and answering phone calls. This keeps me busy. | At noon, I take a break to drink coffee and chat with colleagues. It is a nice time. | In the afternoon, I focus on scheduling meetings. Sometimes, I take a walk to relax. | ./out_folder/row_1/1.mp3 | ./out_folder/row_1/2.mp3 | ./out_folder/row_1/3.mp3 | ./out_folder/row_1/4.mp3 | half past eight am, seven am                      | checking emails, answering phone calls                  | noon, drink coffee, chat with colleagues           | scheduling meetings, take a walk   |
        | 2  | I start my work at five pm. I sit at my desk and open my laptop. I have many tasks to do. | At a quarter to six pm, I take a short break. I drink some water and stretch my legs. | When my tasks are completed, I feel happy. I know I have done a good job today. | I feel satisfied and accomplished. It was a productive day at work. | ./out_folder/row_2/1.mp3 | ./out_folder/row_2/2.mp3 | ./out_folder/row_2/3.mp3 | ./out_folder/row_2/4.mp3 | at five pm, desk, laptop                           | at a quarter to six pm, break, water                     | when my tasks are completed, happy, job             | satisfied, accomplished, productive |

    - Tham số: 
        - processing_func là 1 hàm: ... Khi gọi processing_func(paragraph_dict)
    - Prompt: 
        - gen df1
            - Từ file_path, tìm đến sheet_name, 
            - Duyệt qua từng hàng: 
                - paragraph1 = # get data từ cột content_part_1
                - list1 = # get data từ cột vocab_part_1
                - Chạy ex1_DIEN_VAO_CHO_TRONG_Dang1Listening(paragraph1, list1)
            -   data.append({
                "question": response_dict["question"],
                "paragraph": paragraph_text,  # paragraph_dict.get("paragraph1, 2, 3, 4"),     # paragraph1, 2, 3, 4 tương ứng các đoạn luôn
                "answer": "\n".join([answer.strip() for answer in response_dict.get("answers_list", [])])
            - Trả ra được df1

        - Tương tự df2, 3, 4    
        })
    """

    """
    Process the Excel file to generate content and audio based on the specified processing function.

    Args:
        file_path (str): Path to the input Excel file.
        sheet_name (str): Name of the sheet to process.
        processing_func (callable): Function to process each paragraph.
        content_col (str): Name of the column containing content parts.
        vocab_col (str): Name of the column containing vocabulary parts.

    Returns:
        pd.DataFrame: DataFrame containing processed data.
    """
    # Load the Excel file
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Initialize list to store data
    data = []
    
    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        paragraph_text = row[content_col]
        vocab_list = row[vocab_col].split(", ")

        try:
            # Call the processing function
            response_json = processing_func(paragraph_text, vocab_list)
            
            # Debug: print response_json to check its value
            print(f"Response JSON for row {index}: {response_json}")
            
            # Ensure response_json is not empty
            if not response_json:
                print(f"Warning: Processing function returned an empty response for row {index}")
                continue
            
            # Convert JSON string to dictionary
            response_dict = json.loads(response_json)
            
            # Append the result to the data list
            data.append({
                "question": response_dict["question"],
                "paragraph": paragraph_text,
                "answer": "\n".join([answer.strip() for answer in response_dict.get("answers_list", [])])
            })
        except json.JSONDecodeError as e:
            print(f"JSON decode error for row {index}: {e}")
            print(f"Problematic JSON string: {response_json}")
            continue
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            continue
    
    # Create a DataFrame from the data
    output_df = pd.DataFrame(data)
    
    return output_df

# Define the file path and sheet name
file_path = r'D:\OneDrive - Hanoi University of Science and Technology\ITE10-DS&AI-HUST\Learn&Task\PRODUCT_THECOACH\Task2_ContentGeneration\ĐÓNG_GÓI_1_FLOW\output_folder\output_dang1_LISTENING.xlsx'
sheet_name = 'mix_listen_content'



# Define the processing functions (assuming these are defined elsewhere in your code)
def process_paragraph1(paragraph_text, vocab_list):
    return ex1_DIEN_VAO_CHO_TRONG_Dang1Listening(paragraph_text, vocab_list)

def process_paragraph2(paragraph_text, vocab_list):
    return ex2_CHON_DAP_AN_PHU_HOP_Dang1Listening(paragraph_text, vocab_list)

def process_paragraph3(paragraph_text, vocab_list):
    return ex3_NOI_DE_TRA_LOI_Dang1Listening(paragraph_text, vocab_list)

def process_paragraph4(paragraph_text, vocab_list):
    return ex4_DIEN_PHAN_CON_THIEU_Dang1Listening(paragraph_text, vocab_list)

# result_df, output_file_path = input_2_gen_content_AND_audio_2_excel(input_file_path, output_folder)


# Process each type of paragraph and store in DataFrames
df1 = process_excel(file_path, sheet_name, process_paragraph1, 'content_part_1', 'vocab_part_1')
df2 = process_excel(file_path, sheet_name, process_paragraph2, 'content_part_2', 'vocab_part_2')
df3 = process_excel(file_path, sheet_name, process_paragraph3, 'content_part_3', 'vocab_part_3')
df4 = process_excel(file_path, sheet_name, process_paragraph4, 'content_part_4', 'vocab_part_4')

# Write all DataFrames to a single Excel file with different sheets
output_file_path = 'combined_Dang1Listening_output.xlsx'
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    result_df.to_excel(writer, sheet_name='mix_listent_content', index=False)
    print(f"Sheet 'Ex1_listen_fill_choose' has been successfully written.")

    df1.to_excel(writer, sheet_name='Ex1_listen_fill_choose', index=False)
    print(f"Sheet 'Ex1_listen_fill_choose' has been successfully written.")

    df2.to_excel(writer, sheet_name='Ex2_listen_choice', index=False)
    print(f"Sheet 'Ex2_listen_choice' has been successfully written.")
    
    df3.to_excel(writer, sheet_name='Ex3_listen_speak', index=False)
    print(f"Sheet 'Ex3_listen_speak' has been successfully written.")

    df4.to_excel(writer, sheet_name='Ex4_listen_fill_type', index=False)
    print(f"Sheet 'Ex4_listen_fill_type' has been successfully written.")

print(f"Data has been successfully written to {output_file_path}")

Response JSON for row 0: {
    "input_text": "Paragraph 1 about Talk about you working day",
    "selected_term": ["half past eight am", "checking emails"],
    "question": "Paragraph 1 about Talk about you __ day",
    "answers_list": ["working", "emails", "half", "past", "eight", "am", "checking"]
}
Response JSON for row 1: {
    "input_text": "Paragraph 1 about Talk about you working day (2)",
    "selected_term": ["at five pm", "satisfied"],
    "question": "Paragraph 1 about Talk about you working day (2) __ five pm __",
    "answers_list": ["at", "satisfied", "about", "you", "day", "work", "tasks"]
}
Response JSON for row 0: {
    "selected_term": "checking emails",
    "sentence": "I usually start my day by checking emails to see if there are any urgent tasks that need my attention.",
    "question": "What activity does the person usually start their day with?",
    "answers_list": [
        "Checking emails",
        "Scheduling meetings",
        "Answering phone calls",
     